In [56]:
import xmltodict, json, feedparser
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time, re
import pandas as pd
from collections import defaultdict
import json
from datetime import datetime
from datetime import date
from selenium.webdriver.common.by import By
from pprint import pprint
import dataframe_image as dfi
import os, sys
from PIL import Image
import cv2

In [57]:
# 공급계약

title = '단일판매ㆍ공급계약체결'
link = 'http://dart.fss.or.kr/api/link.jsp?rcpNo=20220927900266'
companyName = '에스케이바이오팜'
text = title
date = '2022-09-26'

if ('단일판매ㆍ공급계약체결' in title):  # and not('정정' in title):

  option = Options()
  option.add_argument('headless')
  driver = webdriver.Chrome(options=option)
  driver.get(link)
  # time.sleep(2)
  html_source = driver.page_source

  rcpNo = re.findall(r"openPdfDownload\('(.*?)', '", html_source)
  dcmNo = re.findall(r"', '(.*?)'\);", html_source)

  # url이 있어야 read_html을 통해 table 가져올 수 있다
  frame_url = 'https://dart.fss.or.kr/report/viewer.do?rcpNo=' + rcpNo[0] + '&dcmNo=' + dcmNo[2][0:7] + '&eleId=' + '0' + '&offset=' + '0' + '&length=' + '0' + '&dtd=HTML'

  table_ifrm = pd.read_html(frame_url, index_col=1)

  # print(datetime.now())
  for index, row in table_ifrm[0].iterrows():
    if '계약금액' in index:
      contractAmount = f'{int(row[2]):,}'
      break
  for index, row in table_ifrm[0].iterrows():
    if '매출액' in index:
      latestSales = f'{int(row[2]):,}'
      break
  for index, row in table_ifrm[0].iterrows():
    if '%' in index:
      contractSalesRatio = f'{round(float(row[2]), 1):,}'
      break
  for index, row in table_ifrm[0].iterrows():
    if '상대방' in index:
      counterPart = row[2]
      break
  for index, row in table_ifrm[0].iterrows():
    if '시작' in index:
      beginningDay = row[2]
      break
  for index, row in table_ifrm[0].iterrows():
    if '종료' in index:
      endingDay = row[2]
      break
  
  period
  # period = date.fromisoformat(endingDay).days - date.fromisoformat(beginningDay).days + 1

  # for index, row in table_ifrm[0].iterrows():
  #   if '일시' in index:
  #     if '20' in row[2]:
  #       dateNew = row[2]

  # try:
  #   with open('calendarDB.json', encoding='utf-8') as db:
  #     res = json.load(db)
  #   print(dateNew)
  #   args = [dateNew, companyName, text, link]
  #   res = add_entry(res, *args)
  #   with open('calendarDB.json', 'w', encoding='utf-8') as db:
  #     json.dump(res, db, ensure_ascii=False)
  # except:
  #   pass






AttributeError: 'str' object has no attribute 'fromisoformat'

In [61]:
endingDay

'2023-09-26'

In [60]:
td = date.fromisoformat(endingDay.replace("'",'')) - date.fromisoformat(beginningDay.replace("'",''))
td

AttributeError: 'str' object has no attribute 'fromisoformat'

In [30]:
table_ifrm[0]

,0,2,3
1,,,
1. 판매ㆍ공급계약 내용,1. 판매ㆍ공급계약 내용,IBC101 CAR-T 세포 제조 위수탁 계약,IBC101 CAR-T 세포 제조 위수탁 계약
조건부 계약여부,2. 계약내역,미해당,미해당
확정 계약금액,2. 계약내역,480000000,480000000
조건부 계약금액,2. 계약내역,-,-
계약금액 총액(원),2. 계약내역,480000000,480000000
최근 매출액(원),2. 계약내역,909090909,909090909
매출액 대비(%),2. 계약내역,52.8,52.8
3. 계약상대방,3. 계약상대방,(주)이노베이션바이오,(주)이노베이션바이오
-최근 매출액(원),-최근 매출액(원),0,0
